In [1]:
import requests
import pandas as pd

## Pobieranie wyników turnieju- część 1

In [2]:
def scores_pipeline(url):
    pd.read_html(requests.get(url).content)[-1].to_csv("mydata.csv")
    df=pd.read_csv('mydata.csv')
    df=df.drop(['Unnamed: 0','+/-','pkl', 'pdf'],axis=1)
    return df

In [3]:
def regex_trans_scores(df):
    import re
    pierwszy=[]
    drugi=[]
    for i in range(len(df)):
        pierwszy.append(re.findall('[A-ZŻŹĆĄŚĘŁÓŃ][^A-ZŻŹĆĄŚĘŁÓŃ]*', df.nazwiska[i])[0]+re.findall('[A-ZŻŹĆĄŚĘŁÓŃ][^A-ZŻŹĆĄŚĘŁÓŃ]*', df.nazwiska[i])[1])
        drugi.append(re.findall('[A-ZŻŹĆĄŚĘŁÓŃ][^A-ZŻŹĆĄŚĘŁÓŃ]*', df.nazwiska[i])[2]+re.findall('[A-ZŻŹĆĄŚĘŁÓŃ][^A-ZŻŹĆĄŚĘŁÓŃ]*', df.nazwiska[i])[3])
    df['nazwisko_1']=pierwszy
    df['nazwisko_2']=drugi
    df=df.drop('nazwiska',axis=1)


    wk1=[]
    wk2=[]
    for i in range(len(df)):
        wk1.append(re.findall("\d+\.\d", df.wk[i])[0])
        wk2.append(re.findall("\d+\.\d", df.wk[i])[1])
    df['wk_1']=wk1
    df['wk_2']=wk2
    df=df.drop(['wk','ośrodek','okręg'],axis=1)
    return df

## Pobieranie wyników poszczególnych rozdań

In [4]:
def get_hist(n):
    variable_names = [f"df{i}" for i in range(1,n+1)]
    for i in range(30):
        pd.read_html(requests.get(f"https://r.bridgespider.com/25890/prot/{i+1}/").content)[-1].to_csv("mydat.csv")
        variable_names[i]=pd.read_csv('mydat.csv')
    return variable_names

In [5]:
def pipeline(df2):
    df2=df2.drop(['Unnamed: 0','ośrodek','runda','wist','Unnamed: 11','lew','kontrakt'],axis=1)
    df2['rozg.']=df2['rozg.'].replace(['E','W'],'EW')
    df2['rozg.']=df2['rozg.'].replace(['N','S'],'NS')
    df2['zapis_NS']=df2['zapis']
    df2['zapis_EW']=df2['zapis']*(-1)
    df2=df2.drop('zapis',axis=1)
    return df2

In [6]:
def pipeline2(df_x):
    tmp1=df_x.copy()
    tmp2=df_x.copy()
    tmp1=tmp1.drop(['EW','% EW','zapis_EW'],axis=1)
    tmp1['linia']='NS'
    tmp2=tmp2.drop(['NS','% NS','zapis_NS'],axis=1)
    tmp2['linia']='EW'
    tmp1.columns=['id','rozg','wynik','zapis','linia']
    tmp2.columns=['id','rozg','wynik','zapis','linia']
    tmp=pd.concat([tmp1,tmp2])
    return tmp

In [7]:
def refactor(variable_names):
    for i in range(len(variable_names)):
        variable_names[i]=pipeline(variable_names[i])
        variable_names[i]=pipeline2(variable_names[i])
    return variable_names

## Main

In [8]:
scores=scores_pipeline("https://r.bridgespider.com/25890/")
scores=regex_trans_scores(scores)
scores['nr_turnieju']=25890
scores

,m-ce,nr,%,nazwisko_1,nazwisko_2,wk_1,wk_2,nr_turnieju
0,1,99,66.18,Jerzy Pala,Jan Styczyński,5.0,5.0,25890
1,2,140,65.97,Sebastian Rawlik,Damian Mazurak,5.0,5.0,25890
2,3,10,64.60,Janusz Czerniawski,Zbigniew Buciuto,5.0,3.0,25890
3,4,527,64.47,Ryszard Smejda,Damian Wroński,7.0,11.0,25890
4,5,247,64.31,Wojciech Słysz,Marek Melsztyński,1.5,2.0,25890
...,...,...,...,...,...,...,...,...
517,518,271,33.67,Jerzy Słomski,Jolanta Wiśniewska,2.5,4.0,25890
518,519,356,32.97,Leszek Zieliński,Janusz Bakalarz,4.0,0.5,25890
519,520,361,31.28,Natalia Fręchowicz,Jan Luber,0.0,0.0,25890
520,521,109,31.06,Wincenty Tęgowski,Paweł Jankiewicz,0.0,0.0,25890


In [9]:
history=get_hist(30)
history=refactor(history)
history[0]

,id,rozg,wynik,zapis,linia
0,8,EW,67.46,-150,NS
1,5,EW,57.74,-180,NS
2,10,EW,57.74,-180,NS
3,3,EW,38.89,-400,NS
4,4,EW,38.89,-400,NS
...,...,...,...,...,...
248,532,EW,1.39,-100,EW
249,528,EW,61.11,400,EW
250,523,EW,61.11,400,EW
251,522,EW,86.71,430,EW


## Część 2

In [10]:
for i in range(30):
    history[i]['numer_rozdania']=i+1    

In [11]:
tmp=pd.concat([history[0],history[1]])
for i in range(2,len(history)):
    tmp=pd.concat([tmp,history[i]])
tmp    

,id,rozg,wynik,zapis,linia,numer_rozdania
0,8,EW,67.46,-150.0,NS,1
1,5,EW,57.74,-180.0,NS,1
2,10,EW,57.74,-180.0,NS,1
3,3,EW,38.89,-400.0,NS,1
4,4,EW,38.89,-400.0,NS,1
...,...,...,...,...,...,...
136,531,EW,78.84,170.0,EW,30
137,527,EW,78.84,170.0,EW,30
138,522,EW,78.84,170.0,EW,30
139,521,EW,56.41,140.0,EW,30


In [12]:
tmp1=tmp[tmp['rozg']==tmp['linia']]
tmp2=tmp[tmp['rozg']!=tmp['linia']]
tmp1['czy_rozgrywa']=True
tmp2['czy_rozgrywa']=False
tmp=pd.concat([tmp1,tmp2])

<ipython-input-12-80521d35c651>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['czy_rozgrywa']=True
<ipython-input-12-80521d35c651>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp2['czy_rozgrywa']=False


In [13]:
tmp[tmp['id']==99].reset_index()

,index,id,rozg,wynik,zapis,linia,numer_rozdania,czy_rozgrywa
0,47,99,EW,61.11,400.0,EW,1,True
1,47,99,EW,89.00,1010.0,EW,2,True
2,51,99,NS,69.48,650.0,NS,4,True
3,51,99,NS,58.53,140.0,NS,5,True
4,51,99,NS,82.74,460.0,NS,6,True
5,51,99,EW,98.40,590.0,EW,8,True
6,48,99,NS,54.96,140.0,NS,11,True
7,52,99,EW,96.63,170.0,EW,16,True
8,52,99,EW,88.25,-100.0,EW,18,True
9,50,99,NS,24.70,180.0,NS,21,True


In [14]:
tab=[25845,25809,25773,25737,25692,24384,24339,24303,24267,24222,24186,24150,22079 ,22043 ,21800,21764,21728,17693,17666]

In [15]:
url="https://r.bridgespider.com/"

In [16]:
url+str(tab[2])+'/'

'https://r.bridgespider.com/25773/'

In [17]:
scores

,m-ce,nr,%,nazwisko_1,nazwisko_2,wk_1,wk_2,nr_turnieju
0,1,99,66.18,Jerzy Pala,Jan Styczyński,5.0,5.0,25890
1,2,140,65.97,Sebastian Rawlik,Damian Mazurak,5.0,5.0,25890
2,3,10,64.60,Janusz Czerniawski,Zbigniew Buciuto,5.0,3.0,25890
3,4,527,64.47,Ryszard Smejda,Damian Wroński,7.0,11.0,25890
4,5,247,64.31,Wojciech Słysz,Marek Melsztyński,1.5,2.0,25890
...,...,...,...,...,...,...,...,...
517,518,271,33.67,Jerzy Słomski,Jolanta Wiśniewska,2.5,4.0,25890
518,519,356,32.97,Leszek Zieliński,Janusz Bakalarz,4.0,0.5,25890
519,520,361,31.28,Natalia Fręchowicz,Jan Luber,0.0,0.0,25890
520,521,109,31.06,Wincenty Tęgowski,Paweł Jankiewicz,0.0,0.0,25890


-------------------------------

In [18]:
for i in tab:
    url="https://r.bridgespider.com/"
    url=url+str(i)+'/'
    tmp=scores_pipeline(url)
    tmp=regex_trans_scores(tmp)
    tmp['nr_turnieju']=i
    scores=pd.concat([tmp,scores])
    
scores

,m-ce,nr,%,nazwisko_1,nazwisko_2,wk_1,wk_2,nr_turnieju
0,1,660,67.09,Grażyna Busse,Piotr Busse,15.0,19.0,17666
1,2,597,65.96,Jerzy Ratkowski,Jerzy Sukow,5.0,12.0,17666
2,3,374,65.35,Dariusz Krakowiak,Jacek Lech,3.0,2.0,17666
3,4,720,64.69,Władysław Tomasiak,Wojciech Bondarewicz,13.0,4.0,17666
4,5,33,64.43,Jarosław Kowalik,Roman Wyszowski,5.0,5.0,17666
...,...,...,...,...,...,...,...,...
517,518,271,33.67,Jerzy Słomski,Jolanta Wiśniewska,2.5,4.0,25890
518,519,356,32.97,Leszek Zieliński,Janusz Bakalarz,4.0,0.5,25890
519,520,361,31.28,Natalia Fręchowicz,Jan Luber,0.0,0.0,25890
520,521,109,31.06,Wincenty Tęgowski,Paweł Jankiewicz,0.0,0.0,25890


In [19]:
scores['unique_id'] = scores.groupby(['nazwisko_1', 'nazwisko_2']).ngroup()
scores

,m-ce,nr,%,nazwisko_1,nazwisko_2,wk_1,wk_2,nr_turnieju,unique_id
0,1,660,67.09,Grażyna Busse,Piotr Busse,15.0,19.0,17666,1199
1,2,597,65.96,Jerzy Ratkowski,Jerzy Sukow,5.0,12.0,17666,2029
2,3,374,65.35,Dariusz Krakowiak,Jacek Lech,3.0,2.0,17666,883
3,4,720,64.69,Władysław Tomasiak,Wojciech Bondarewicz,13.0,4.0,17666,4565
4,5,33,64.43,Jarosław Kowalik,Roman Wyszowski,5.0,5.0,17666,1865
...,...,...,...,...,...,...,...,...,...
517,518,271,33.67,Jerzy Słomski,Jolanta Wiśniewska,2.5,4.0,25890,2078
518,519,356,32.97,Leszek Zieliński,Janusz Bakalarz,4.0,0.5,25890,2591
519,520,361,31.28,Natalia Fręchowicz,Jan Luber,0.0,0.0,25890,3234
520,521,109,31.06,Wincenty Tęgowski,Paweł Jankiewicz,0.0,0.0,25890,4430


In [20]:
vector=scores["unique_id"].value_counts().index.tolist()[:10]

In [21]:
top10=scores.loc[scores["unique_id"].isin(vector)]

In [22]:
top10

,m-ce,nr,%,nazwisko_1,nazwisko_2,wk_1,wk_2,nr_turnieju,unique_id
55,56,700,59.53,Rafał Junik,Jolanta Ziętara,7.0,5.0,17666,3473
148,149,592,55.67,Wiesław Pietrzak,Zenon Wawer,5.0,5.0,17666,4382
237,238,207,52.96,Krzysztof Borowik,Henryk Choiński,0.0,3.0,17666,2359
250,251,202,52.59,Franciszek Bułyszko,Zbigniew Józefiak,4.0,4.0,17666,1158
318,319,13,51.25,Andrzej Augustyniak,Henryk Gayny,5.0,2.0,17666,178
...,...,...,...,...,...,...,...,...,...
193,194,388,52.54,Wiesław Pietrzak,Zenon Wawer,5.0,5.0,25890,4382
285,286,198,49.09,Krzysztof Borowik,Henryk Choiński,1.5,3.0,25890,2359
405,406,200,44.91,Sławomir Rutecki,Tadeusz Birecki,7.0,5.0,25890,3963
441,442,199,43.00,Franciszek Bułyszko,Zbigniew Józefiak,4.0,5.0,25890,1158


## Zawodnicy z Cezara

In [42]:
url="https://msc.com.pl/cezar/?p=51&rok=0&page=1"

In [32]:
pd.read_html(requests.get(url).content)[-1].to_csv("mydata.csv")
df=pd.read_csv('mydata.csv')

In [44]:
tables = pd.read_html(url,match="klub / drużyna")
df=tables[2]

There are :  3  tables
Take look at table 0


In [48]:
for i in range(65,337):
    url=f"https://msc.com.pl/cezar/?p=51&rok=0&page={i}"
    tables = pd.read_html(url,match="klub / drużyna")
    df=pd.concat([df,tables[2]])

In [51]:
df.to_csv('Lista_Zawodnikow.csv')

In [52]:
scores

,m-ce,nr,%,nazwisko_1,nazwisko_2,wk_1,wk_2,nr_turnieju,unique_id
0,1,660,67.09,Grażyna Busse,Piotr Busse,15.0,19.0,17666,1199
1,2,597,65.96,Jerzy Ratkowski,Jerzy Sukow,5.0,12.0,17666,2029
2,3,374,65.35,Dariusz Krakowiak,Jacek Lech,3.0,2.0,17666,883
3,4,720,64.69,Władysław Tomasiak,Wojciech Bondarewicz,13.0,4.0,17666,4565
4,5,33,64.43,Jarosław Kowalik,Roman Wyszowski,5.0,5.0,17666,1865
...,...,...,...,...,...,...,...,...,...
517,518,271,33.67,Jerzy Słomski,Jolanta Wiśniewska,2.5,4.0,25890,2078
518,519,356,32.97,Leszek Zieliński,Janusz Bakalarz,4.0,0.5,25890,2591
519,520,361,31.28,Natalia Fręchowicz,Jan Luber,0.0,0.0,25890,3234
520,521,109,31.06,Wincenty Tęgowski,Paweł Jankiewicz,0.0,0.0,25890,4430


In [53]:
scores.columns

Index(['m-ce', 'nr', '%', 'nazwisko_1', 'nazwisko_2', 'wk_1', 'wk_2',
       'nr_turnieju', 'unique_id'],
      dtype='object')

In [67]:
scores_1=scores[['m-ce', 'nr', '%', 'nazwisko_1','wk_1','nr_turnieju']]
scores_2=scores[['m-ce', 'nr', '%', 'nazwisko_2','wk_2','nr_turnieju']]

In [60]:
import warnings
warnings.filterwarnings('ignore')

In [68]:
scores_1['nazwisko']=scores_1['nazwisko_1']
scores_2['nazwisko']=scores_2['nazwisko_2']
scores_1['wk']=scores_1['wk_1']
scores_2['wk']=scores_2['wk_2']
scores_1=scores_1.drop(['nazwisko_1','wk_1'],axis=1)
scores_2=scores_2.drop(['nazwisko_2','wk_2'],axis=1)

In [69]:
scores_new=pd.concat([scores_1,scores_2])

In [84]:
df.rename(columns = {2: 'name'}, inplace = True)

In [87]:
df.rename(columns = {1: 'PID'}, inplace = True)

In [158]:
df

,0,PID,name,3,4,5,6,7,8
0,lokata,PID,imię i nazwisko,WZBS,WK,TK,PKL,liga,klub / drużyna
1,1.,06706,Apolinary Kowalski,WM,24,AS,187 649,E,VITAS ESBS Elblag
2,2.,08513,Piotr Gawryś,DS,24,AS,149 944,E,BRIDGESCANNER KS AZS Wratislavia I WR
3,3.,06658,Jacek Kalita,SW,24,AS,146 928,E,SPS CONSTRUCTION Kielce
4,4.,01221,Cezary Balicki,DS,24,AS,143 379,-,NaN
...,...,...,...,...,...,...,...,...,...
46,NaN,20956,Maciej Wisz,PK,0,D,1,-,Singleton Podkarpacki
47,NaN,17464,Małgorzata Wiśniewska,DS,0,D,1,-,NaN
48,NaN,20096,Adrian Witkowski,KP,0,D,1,-,NaN
49,NaN,21716,Jakub Witowski,PK,0,D,1,-,Singleton Podkarpacki


In [159]:
last_scores=scores_new.merge(df[['PID','name']],left_on='nazwisko',right_on='name')

In [160]:
last_scores=last_scores.drop(['nr','name'],axis=1)

In [161]:
last_scores

,m-ce,%,nr_turnieju,nazwisko,wk,PID
0,1,67.09,17666,Grażyna Busse,15.0,08485
1,44,60.77,17693,Grażyna Busse,15.0,08485
2,105,57.30,21800,Grażyna Busse,15.0,08485
3,103,50.05,22079,Grażyna Busse,15.0,08485
4,312,44.06,24267,Grażyna Busse,15.0,08485
...,...,...,...,...,...,...
19268,303,48.75,25890,Andrzej Grześko,2.0,19403
19269,321,48.22,25890,Tymoteusz Zaborowski,1.5,17787
19270,349,47.28,25890,Tadeusz Ciechanowski,5.0,05587
19271,467,40.96,25890,Wiesław Kulikowski,1.5,15109


In [162]:
zz=last_scores[last_scores[['nr_turnieju','nazwisko']].duplicated(keep=False)]

In [163]:
last_scores=pd.concat([last_scores,zz]).drop_duplicates(keep=False)

In [164]:
last_scores=tmp.merge(last_scores,left_on=['nr_turnieju','PID'],right_on=['nr_turnieju','PID'])

In [166]:
last_scores

,nazwisko_x,nr_turnieju,PID,m-ce,%,nazwisko_y,wk
0,Ada Szewc,25737,21071,206,50.92,Ada Szewc,1.5
1,Adam Amrozy,24267,06313,124,52.41,Adam Amrozy,4.0
2,Adam Amrozy,24303,06313,118,54.81,Adam Amrozy,4.0
3,Adam Amrozy,24339,06313,248,49.34,Adam Amrozy,4.0
4,Adam Amrozy,25773,06313,132,54.41,Adam Amrozy,4.0
...,...,...,...,...,...,...,...
17037,Łukasz Wieczorek,25773,13084,123,54.95,Łukasz Wieczorek,3.0
17038,Łukasz Wieczorek,25809,13084,119,55.42,Łukasz Wieczorek,3.0
17039,Łukasz Wieczorek,25845,13084,383,45.45,Łukasz Wieczorek,3.0
17040,Łukasz Wieczorek,25890,13084,335,47.85,Łukasz Wieczorek,3.0


In [167]:
last_scores['nazwisko_x'].value_counts()

Zbigniew Bobak         20
Przemysław Wojtczak    20
Dariusz Krakowiak      20
Stanisław Janiak       20
Andrzej Kędzierski     20
                       ..
Mirosław Samujłło       1
Karol Kamas             1
Leszek Kita             1
Sebastian Zimorski      1
Ryszard Gmoch           1
Name: nazwisko_x, Length: 2987, dtype: int64